In [3]:
! du -h ../data/processed/pagecount

415M	../data/processed/pagecount


In [4]:
pagecount = spark.read.parquet("../data/processed/pagecount")

In [6]:
pagecount.show(n=5)

+-------+-------------------+-----+
|page_id|         visit_time|count|
+-------+-------------------+-----+
|    698|2019-02-05 00:00:00|   40|
|    698|2019-02-05 01:00:00|   35|
|    698|2019-02-05 02:00:00|   32|
|    698|2019-02-05 03:00:00|   32|
|    698|2019-02-05 04:00:00|   29|
+-------+-------------------+-----+
only showing top 5 rows



In [14]:
import pyspark.sql.functions as F
daily = (
    pagecount
    .select("page_id", F.date_format("visit_time", "yyyy-MM-dd").alias("date"), "count")
    .groupBy("page_id", "date")
    .agg(F.sum("count").alias("count"))
)
daily.show()

+-------+----------+-----+
|page_id|      date|count|
+-------+----------+-----+
|   8651|2019-01-06| 2495|
|   9087|2019-02-23|  274|
|   9087|2019-01-07|  326|
|   9087|2019-01-29|  371|
|   9087|2019-01-06|  215|
|   9417|2019-01-14|  561|
|  10911|2019-02-06|  512|
|  11127|2019-02-21| 1642|
|  12539|2019-01-06|  251|
|  16550|2019-02-26| 1587|
|  16794|2019-01-20|  320|
|  17068|2019-01-21|  859|
|  19036|2019-01-05| 1006|
|  20003|2019-02-23|  346|
|  20003|2019-02-10|  350|
|  22304|2019-02-21|  821|
|  22304|2019-02-15|  576|
|  24614|2019-02-20|  152|
|  25321|2019-01-22|  109|
|  27930|2019-02-08|  136|
+-------+----------+-----+
only showing top 20 rows



In [15]:
daily.repartitionByRange(1, "page_id", "date").write.parquet("../data/enwiki/pagecount_daily_v1")

In [ ]:
! tree -h .